## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [1]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [2]:
# View the column names in the data
print(df.columns)


Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')


In [3]:
# Use the describe function to gather some basic statistics
df.describe()


,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [4]:
# Use this space to do any additional research
# and familiarize yourself with the data.



In [5]:
# What three item categories had the most entries?
print(df['category'].value_counts().head(3))


category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64


In [6]:
# For the category with the most entries,
# which subcategory had the most entries?
print(df[df['category'] == 'consumables']['subcategory'].value_counts().head(1))


subcategory
bathroom supplies    6424
Name: count, dtype: int64


In [7]:
# Which five clients had the most entries in the data?
print(df['client_id'].value_counts().head(5))

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64


In [8]:
# Store the client ids of those top 5 clients in a list.
top_client_ids = df['client_id'].value_counts().head(5).index.tolist()

print(top_client_ids)


[33615, 66037, 46820, 38378, 24741]


In [9]:
# How many total units (the qty column) did the
# client with the most entries order order?
total_units = df[df['client_id'] == top_client_ids[0]]['qty'].sum()

print(total_units)


64313


## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [10]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df['line_subtotal'] = df['unit_price'] * df['qty']

df['line_subtotal'].head()


0    115164.00
1       523.95
2       527.28
3      1056.18
4      3902.00
Name: line_subtotal, dtype: float64

In [11]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
df['total_weight'] = df['unit_weight'] * df['qty']

def calculate_shipping_price(row):
    if row['total_weight'] > 50:
        return row['total_weight'] * 7
    else:
        return row['total_weight'] * 10

df['shipping_price'] = df.apply(calculate_shipping_price, axis=1)

df['shipping_price'].head()


0    5512.50
1     312.90
2     458.64
3     356.70
4    6500.20
Name: shipping_price, dtype: float64

In [12]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%
df['total_price'] = ((df['line_subtotal'] + df['shipping_price']) * 1.0925).round(2)

df['total_price'].head()


0    131839.08
1       914.26
2      1077.12
3      1543.57
4     11364.40
Name: total_price, dtype: float64

In [13]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).
df['line_cost'] = (df['unit_cost'] * df['qty'] + df['shipping_price']).round(2)

df.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,total_price,line_cost
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,36.42,24.85,1.23,29,3,1056.18,35.67,356.70,1543.57,1077.35
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,195.10,108.17,46.43,20,1,3902.00,928.60,6500.20,11364.40,8663.60


In [14]:
# Create a column for the profit of
# each line using line cost and line price
df['line_profit'] = (df['total_price'] - df['line_cost']).round(2)

df.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,total_price,line_cost,line_profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05,46242.03
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79,284.47
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18,311.94
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,24.85,1.23,29,3,1056.18,35.67,356.70,1543.57,1077.35,466.22
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,108.17,46.43,20,1,3902.00,928.60,6500.20,11364.40,8663.60,2700.80


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [15]:
# Check your work using the totals above
order_id = 2742071
check_price = df.loc[df['order_id'] == order_id, 'total_price'].sum()

print(check_price.round(2))

order_id = 2173913
check_price = df.loc[df['order_id'] == order_id, 'total_price'].sum()

print(check_price.round(2))

order_id = 6128929
check_price = df.loc[df['order_id'] == order_id, 'total_price'].sum()

print(check_price.round(2))

152811.89
162388.71
923441.25


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [16]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.
print(df.groupby('client_id')['total_price'].sum().loc[top_client_ids])



client_id
33615     8377308.52
66037    10259514.79
46820     9743794.36
38378    12906550.87
24741    82268892.02
Name: total_price, dtype: float64


In [17]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.


# Group by client_id and aggregate necessary information, with defined column names
summary_df = df.groupby('client_id').agg(
    Units_Purchased=('qty', 'sum'),
    Shipping_Price=('shipping_price', 'sum'),
    Revenue=('total_price', 'sum'),
    Profit=('line_profit', 'sum')
)

# Get the top 5 clients by total profit
top_5_clients = summary_df.nlargest(5, 'Profit')

# Display the summary DataFrame for the top 5 clients
top_5_clients



,Units_Purchased,Shipping_Price,Revenue,Profit
client_id,,,,
78965,4033302,5340756.88,6.559237e+08,2.473528e+08
85931,644423,10122549.19,2.374170e+08,1.092941e+08
38473,402770,4391003.73,1.835526e+08,4.335173e+07
86721,1010426,64335727.72,1.812663e+08,4.297470e+07
24741,239862,5126448.37,8.226889e+07,3.657999e+07


In [18]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.

# Rename all the columns
top_5_clients = top_5_clients.rename(columns={
    'Shipping_Price': 'Total Shipping Price',
    'Revenue': 'Total Revenue',
    'Profit': 'Total Profit',
    'Units_Purchased': 'Total Units Purchased'
})

# Convert the values to millions of dollars
top_5_clients['Total Shipping Price'] = (top_5_clients['Total Shipping Price']/1000000).round(1)
top_5_clients['Total Revenue'] = (top_5_clients['Total Revenue']/1000000).round(1)
top_5_clients['Total Profit'] = (top_5_clients['Total Profit']/1000000).round(1)

# Sort by the total profit from highest to lowest
top_5_clients = top_5_clients.sort_values('Total Profit', ascending=False)

# Make the data more readable
top_5_clients['Total Profit'] = top_5_clients['Total Profit'].apply(lambda x: f'${x}M')
top_5_clients['Total Revenue'] = top_5_clients['Total Revenue'].apply(lambda x: f'${x}M')
top_5_clients['Total Shipping Price'] = top_5_clients['Total Shipping Price'].apply(lambda x: f'${x}M')

top_5_clients.head()



,Total Units Purchased,Total Shipping Price,Total Revenue,Total Profit
client_id,,,,
78965,4033302,$5.3M,$655.9M,$247.4M
85931,644423,$10.1M,$237.4M,$109.3M
38473,402770,$4.4M,$183.6M,$43.4M
86721,1010426,$64.3M,$181.3M,$43.0M
24741,239862,$5.1M,$82.3M,$36.6M
